# SETUP

## Import Libraries

In [ ]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [ ]:
client = MongoClient()
db = client["test"]

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [ ]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

### Initialize Output DF

In [ ]:
matchapps_output_df = pd.DataFrame()

## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [ ]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [ ]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [ ]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [ ]:
matchapps_output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
matchapps_output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
matchapps_output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

#### Auto Amp Notes Average

In [ ]:
matchapps_output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [ ]:
matchapps_output_df["Auto Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    matchapps_output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes

#### Auto All Notes Average

In [ ]:
matchapps_output_df["Auto Notes Avg"] = (matchapps_output_df["Auto Notes Amp Avg"] +
                               matchapps_output_df["Auto Speaker Notes Near Avg"] +
                               matchapps_output_df["Auto Speaker Notes Mid Avg"] +
                               matchapps_output_df["Auto Speaker Notes Far Avg"]) / 4

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [ ]:
matchapps_output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
matchapps_output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
matchapps_output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

#### Teleop Amp Notes Average

In [ ]:
matchapps_output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

#### Teleop All Notes Average

In [ ]:
matchapps_output_df["Tele Notes Avg"] = (matchapps_output_df["Tele Notes Amp Avg"] +
                               matchapps_output_df["Tele Speaker Notes Near Avg"] +
                               matchapps_output_df["Tele Speaker Notes Mid Avg"] +
                               matchapps_output_df["Tele Speaker Notes Far Avg"]) / 4

#### Teleop Percent of Missed Notes

In [ ]:
matchapps_output_df["Tele Percent of Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    matchapps_output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes

### Other Data Analysis Calculations

#### Trap Notes

In [ ]:
matchapps_output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [ ]:
matchapps_output_df["Percent of Successful Climbs"] = None
matchapps_output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        matchapps_output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        matchapps_output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    matchapps_output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

In [ ]:
matchapps_output_df["Harmonized Climbs Total"] = 0

for match in range(1, match_range + 1):
    match_number_df = matchapps_df['metadata.matchNumber'] == match
    robots = matchapps_df[match_number_df]
    robots.sort_values("metadata.robotPosition", inplace=True)
    
    teams_harmonized = []
    
    for i in range(6):
        teams_harmonized.append([robots.iloc[i]["metadata.robotTeam"], robots.iloc[i]["climb"], False])

    teams_harmonized[0][2], teams_harmonized[1][2], teams_harmonized[2][2] = check_team_harmonization_in_alliance(teams_harmonized[0][1], teams_harmonized[1][1], teams_harmonized[2][1])
    teams_harmonized[3][2], teams_harmonized[4][2], teams_harmonized[5][2] = check_team_harmonization_in_alliance(teams_harmonized[3][1], teams_harmonized[4][1], teams_harmonized[5][1])

    for checked_team in teams_harmonized:
        if checked_team[2]:
            matchapps_output_df.at[checked_team[0], "Harmonized Climbs Total"] += 1

#### Percent of Harmonized Climbs out of Successful Climbs

In [ ]:
matchapps_output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

for team in team_list:
    matchapps_output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = matchapps_output_df.at[team, "Harmonized Climbs Total"] / matchapps_output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [ ]:
matchapps_output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [ ]:
matchapps_output_df["Endgame Custom Metric"] = ((matchapps_output_df["Percent of Successful Climbs"] * 5) ** 2 + (matchapps_output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (matchapps_output_df["Trap Notes Avg"] * 5) ** 2) / 3
matchapps_output_df["Endgame Custom Metric"] = matchapps_output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

## Matchapps Finalize and Send Data

### Send Data

In [ ]:
matchapps_output_df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [ ]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

### Initialize Output DF

In [ ]:
superapps_output_df = pd.DataFrame()

## Superapps Data Analysis

### Fouls

In [ ]:
superapps_output_df["Foul - In Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
superapps_output_df["Foul - Damage Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
superapps_output_df["Foul - Tip Entangle Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
superapps_output_df["Foul - Pin Bot"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
superapps_output_df["Foul - Podium"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
superapps_output_df["Foul - Zone"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
superapps_output_df["Foul - Stage"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
superapps_output_df["Foul - Over Exc Chute"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

superapps_output_df["Average Fouls per Match"] = None

for team in team_list:
    superapps_output_df.at[team, "Average Fouls per Match"] = superapps_output_df.at[team, "Foul - In Bot"] + superapps_output_df.at[team, "Foul - Damage Bot"] + superapps_output_df.at[team, "Foul - Tip Entangle Bot"] + superapps_output_df.at[team, "Foul - Pin Bot"] + superapps_output_df.at[team, "Foul - Podium"] + superapps_output_df.at[team, "Foul - Zone"] + superapps_output_df.at[team, "Foul - Stage"] + superapps_output_df.at[team, "Foul - Over Exc Chute"]

### Defense

In [ ]:
superapps_output_df["Full Defense Total"] = superapps_df[superapps_df["defense"] == "fullDef"].groupby("metadata.robotTeam").size()
superapps_output_df["Some Defense Total"] = superapps_df[superapps_df["defense"] == "someDef"].groupby("metadata.robotTeam").size()
superapps_output_df["No Defense Total"] = superapps_df[superapps_df["defense"] == "noDef"].groupby("metadata.robotTeam").size()

## Superapps Finalize and Send Data

### Send Data

In [ ]:
superapps_output_df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)